
Финальный проект
--

Вариант 2
--

Луговских Ирина
--

___

**Задание 2. SQL**

**2.1 Очень усердные ученики.**

**2.1.1 Условие**

Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. Каждое такое маленькое задание называется "горошиной".

Назовём очень усердным учеником того пользователя, который хотя бы раз за текущий месяц правильно решил 20 горошин.

**2.1.2 Задача**

Дана таблица default.peas:

|Название атрибута||Тип атрибута||Смысловое значение|
|---||---||---|
|st_id||int||ID ученика|
|timest||timestamp||Время решения карточки|
|correct||bool||Правильно ли решена горошина?|
|subject||text||Дисциплина в которой находится горошина|


Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов.NB! Под усердным студентом мы понимаем студента, который правильно решил 20 задач за текущий месяц.

In [3]:
import pandas as pd
import pandahouse as ph

In [4]:
#объявляем параметры подключения
connection_default = {'host': 'https://clickhouse.lab.karpov.courses',
                      'database':'default',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

In [5]:
#функция, выводит на экран размер таблицы, количество уникальный и пропущенных значений для каждого столбца
def preliminary_analysis (df):
    print(f'Размер таблицы {df.shape}\n')
    print(f'''Уникальные значения 
{df.nunique().to_string()}\n''')
    print(f'''Пропущенные значения 
{df.isna().sum().to_string()}''')

Посмотрим на таблицу peas

In [6]:
query = '''
SELECT st_id,
    CAST (timest AS DATETIME) AS timest,
    correct, 
    subject
FROM peas
'''

peas = ph.read_clickhouse(query, connection=connection_default)
peas.head(3)

,st_id,timest,correct,subject
0,100379,2021-10-30 13:32:29,1,Theory of probability
1,100379,2021-10-30 14:11:19,0,Vizualization
2,100379,2021-10-30 15:54:22,1,Theory of probability


In [7]:
preliminary_analysis(peas)

Размер таблицы (13245, 4)

Уникальные значения 
st_id        308
timest     12238
correct        2
subject        5

Пропущенные значения 
st_id      0
timest     0
correct    0
subject    0


Посмотрим, данные за какие даты есть в таблице

In [8]:
query = '''
SELECT MIN(CAST (timest AS DATETIME)) AS min_date_time,
    MAX(CAST (timest AS DATETIME)) AS max_date_time
FROM peas
'''

ph.read_clickhouse(query, connection=connection_default)

,min_date_time,max_date_time
0,2021-10-30 13:00:00,2021-10-31 17:29:52


В таблице данные чуть больше чем за одни сутки, к тому же все записи относятся к одному месяцу, поэтому дополнительная фильтрация по дате не требуется

Переходим к решению задачи

In [9]:
query = '''
SELECT COUNT(st_id) AS COUNT
FROM 
(
    SELECT st_id,
        SUM(correct) AS correct
    FROM peas
    GROUP BY st_id
    HAVING correct >= 20
)
'''

ph.read_clickhouse(query, connection=connection_default)

,COUNT
0,136


**Ответ:** 136 очень усердных студентов.

___

**2.2 Оптимизация воронки**

**2.2.1 Условие**

Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.

**2.2.2 Задача**

Дана таблицы: 

*default.peas:*

|Название атрибута||Тип атрибута||Смысловое значение|
|---||---||---|
|st_id||int||ID ученика|
|timest||timestamp||Время решения карточки|
|correct||bool||Правильно ли решена горошина?|
|subject||text||Дисциплина в которой находится горошина|

*default.studs:*

|Название атрибута||Тип атрибута||Смысловое значение|
|---||---||---|
|st_id||int||ID ученика|
|test_grp||text||Метка ученика в данном эксперименте|

и *default.final_project_check:*

|Название атрибута||Тип атрибута||Смысловое значение|
|---||---||---|
|st_id||int||ID ученика|
|sale_time||timestamp||Время покупки|
|money||int||Цена, по которой приобрели данный курс|
|subject||text||Дисциплина в которой находится горошина| 

Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:

 - ARPU 
 - ARPAU 
 - CR в покупку 
 - СR активного пользователя в покупку 
 - CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике

ARPU считается относительно всех пользователей, попавших в группы.

Активным считается пользователь, за все время решивший больше 10 задач правильно в любых дисциплинах.

Активным по математике считается пользователь, за все время решивший 2 или больше задач правильно по математике.

Таблицу *peas* мы видели в прошлом задании, теперь посмотрим на таблицы *studs* и *final_project_check*

In [10]:
query = '''
SELECT * 
FROM studs
'''

studs = ph.read_clickhouse(query, connection=connection_default)
studs.head(3)

,st_id,test_grp
0,100379,pilot
1,101432,control
2,104818,pilot


In [11]:
preliminary_analysis(studs)

Размер таблицы (600, 2)

Уникальные значения 
st_id       600
test_grp      2

Пропущенные значения 
st_id       0
test_grp    0


In [12]:
query = '''
SELECT st_id, 
    CAST (sale_time AS DATETIME) AS sale_time,
    money,
    subject
FROM final_project_check
'''

final_project_check = ph.read_clickhouse(query, connection=connection_default)
final_project_check.head(3)

,st_id,sale_time,money,subject
0,101432,2021-10-31 04:44:32,85000,Math
1,101432,2021-10-31 12:43:50,65000,Vizualization
2,104885,2021-10-30 17:05:55,65000,Vizualization


In [13]:
preliminary_analysis(final_project_check)

Размер таблицы (61, 4)

Уникальные значения 
st_id        47
sale_time    61
money         5
subject       5

Пропущенные значения 
st_id        0
sale_time    0
money        0
subject      0


___

Для того, что бы посчитать нужные метрики нам нужно знать:
 - количество решенных горошин
 - количество решенных горошин по математике
 - факт покупки (0/1)
 - факт покупки курса по математике (0/1)
 - сумма всех покупок
 - сумма всех покупок курсов по математике
для каждого пользователя.

Эту информацию мы сможем получить из таблиц *peas* и *final_project_check*

In [14]:
query = '''
SELECT st_id,
    COUNT(correct) AS sum_peas, 
    countIf(correct, subject = 'Math') AS sum_peas_math
FROM peas
GROUP BY st_id
'''

peas_by_id = ph.read_clickhouse(query, connection=connection_default)
peas_by_id.head(3)

,st_id,sum_peas,sum_peas_math
0,414769,40,0
1,878110,1,0
2,700123,237,0


In [15]:
query = '''
SELECT st_id,
    (COUNT(st_id) > 0) AS purchases,
    (countIf(st_id, subject = 'Math') > 0) AS purchases_math,
    SUM(money) AS sum_money,
    sumIf(money, subject = 'Math') AS sum_money_math
FROM final_project_check
GROUP BY st_id
'''

final_project_check_by_id = ph.read_clickhouse(query, connection=connection_default)
final_project_check_by_id.head(3)

,st_id,purchases,purchases_math,sum_money,sum_money_math
0,270197,1,1,225000,85000
1,287212,1,0,135000,0
2,619784,1,0,100000,0


Соберем все данные в одну таблицу, для этого присоединим к таблице *studs* таблицы, полученные в предыдущих шагах.

In [16]:
query = '''
SELECT *
FROM
    studs AS a
FULL JOIN
    (SELECT st_id,
        COUNT(correct) AS sum_peas, 
        countIf(correct, subject = 'Math') AS sum_peas_math
    FROM peas
    GROUP BY st_id
    ) AS b
ON a.st_id = b.st_id
FULL JOIN
    (SELECT st_id,
        (COUNT(st_id) > 0) AS purchases,
        (countIf(st_id, subject = 'Math') > 0) AS purchases_math,
        SUM(money) AS sum_money,
        sumIf(money, subject = 'Math') AS sum_money_math
    FROM final_project_check
    GROUP BY st_id
    ) AS c
ON a.st_id = c.st_id
'''
ph.read_clickhouse(query, connection=connection_default)

,a.st_id,a.test_grp,b.st_id,b.sum_peas,b.sum_peas_math,c.st_id,c.purchases,c.purchases_math,c.sum_money,c.sum_money_math
0,100379,pilot,100379,47,0,,0,0,0,0
1,101432,control,101432,28,0,101432,1,1,150000,85000
2,104818,pilot,,0,0,,0,0,0,0
3,104885,pilot,104885,10,0,104885,1,0,140000,0
4,104966,pilot,104966,47,0,,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
595,988626,pilot,,0,0,,0,0,0,0
596,993249,pilot,,0,0,,0,0,0,0
597,994983,pilot,994983,91,30,,0,0,0,0
598,996911,control,996911,35,0,,0,0,0,0


Мы получили таблицу, в которой для каждого студента есть вся нужная нам информация.

Осталось рассчитать метрики

In [17]:
query = '''
SELECT
    a.test_grp AS group,
    ROUND(sum(c.sum_money) / count(a.st_id)) as ARPU,
    ROUND(sumIf(c.sum_money, b.sum_peas > 10) / countIf(a.st_id, b.sum_peas > 10)) as ARPAU,
    ROUND(sum(c.purchases) / count(a.st_id), 4) as CR,
    ROUND(sumIf(c.purchases, b.sum_peas > 10) / countIf(a.st_id, b.sum_peas > 10), 4) as CR_activ,
    ROUND(sumIf(c.purchases_math, b.sum_peas_math > 2) / countIf(a.st_id, b.sum_peas_math > 2), 4) as CR_activ_math    
FROM
(SELECT *
FROM
    studs AS a
FULL JOIN
    (SELECT st_id,
        COUNT(correct) AS sum_peas, 
        countIf(correct, subject = 'Math') AS sum_peas_math
    FROM peas
    GROUP BY st_id
    ) AS b
ON a.st_id = b.st_id
FULL JOIN
    (SELECT st_id,
        COUNT(st_id) AS purchases,
        countIf(st_id, subject = 'Math') AS purchases_math,
        SUM(money) AS sum_money,
        sumIf(money, subject = 'Math') AS sum_money_math
    FROM final_project_check
    GROUP BY st_id
    ) AS c
ON a.st_id = c.st_id
)
GROUP BY 
    a.test_grp
'''
metrics = ph.read_clickhouse(query, connection=connection_default)
metrics

,group,ARPU,ARPAU,CR,CR_activ,CR_activ_math
0,control,4541.0,9172.0,0.0590,0.1192,0.060
1,pilot,11508.0,24132.0,0.1458,0.2975,0.093


Несложно увидеть, что все метрики в группе *pilot* выше чем в группе *control*. 

Это задание не предполагает глубокого анализа, поэтому я рассчитаю, во сколько раз отличаются показатели.

In [18]:
metrics = metrics.set_index('group').transpose()
metrics['mertic_up'] = metrics.pilot / metrics.control
metrics

group,control,pilot,mertic_up
ARPU,4541.0000,11508.0000,2.534244
ARPAU,9172.0000,24132.0000,2.631051
CR,0.0590,0.1458,2.471186
CR_activ,0.1192,0.2975,2.495805
CR_activ_math,0.0600,0.0930,1.550000


**Вывод:**

CR для активных студентов по математике показатели отличаются в полтора раза.

Все остальные метрики отличаются примерно в два с половиной раза.

Можно с уверенностью сказать, что внедрение нового экрана оплаты показало хорошие результаты.

In [19]:
pip install pandahouse